![](./figures/Logo.PNG)

# 4 a - Coding hints

---

**Set current working directory and import python modules**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import sys
import seaborn as sns
import matplotlib.dates as mdate
sys.path.append('src/')
from sampling import AAT_sampling # module to perform the input sampling

import HyMod

**Load data, prepare input data**

In [2]:
# Read catchment data
catchment_names = ["Pere Marquette River, MI, USA", "Trout River, BC, Canada", 
                   "Siletz River, OR, USA", "Medina River, TX, USA",
                   "South Branch Potomac River, USA", "Bad River, Wisconsin, USA"]
catchment_name = catchment_names[5]
file_dic = {catchment_names[0]: "camels_04122500", catchment_names[1]: "hysets_10BE007", 
            catchment_names[2]: "camels_14305500", catchment_names[3]: "camels_08178880",
            catchment_names[4]: "camels_01605500", catchment_names[5]: "camels_04027000"}
df_obs = pd.read_csv(f"data/{file_dic[catchment_name]}.csv")
# Make sure the date is interpreted as a datetime object -> makes temporal operations easier
df_obs.date = pd.to_datetime(df_obs['date'], format='%Y-%m-%d')
# Index frame by date
df_obs.set_index('date', inplace=True)
# Select time frame
start_date = '2005-01-01'
end_date = '2006-12-31'
df_obs = df_obs[start_date:end_date]
n_days = 365  # days for the model to spin-up (fill with water)
# Reformat the date for plotting
df_obs["date"] = df_obs.index.map(lambda s: s.strftime('%b-%d-%y'))
# Reindex
df_obs = df_obs.reset_index(drop=True)
# Select snow, precip, PET, streamflow and T
df_obs = df_obs[["snow_depth_water_equivalent_mean", "total_precipitation_sum","potential_evaporation_sum","streamflow", "temperature_2m_mean", "date"]]
# Rename variables
df_obs.columns = ["Snow [mm/day]", "P [mm/day]", "PET [mm/day]", "Q [mm/day]", "T [C]", "Date"]

# Extract time series
P = df_obs["P [mm/day]"].to_numpy()
pet = df_obs["PET [mm/day]"].to_numpy()
#temp = df_obs["T [C]"].to_numpy()
Q_obs = df_obs["Q [mm/day]"].to_numpy()

# Number of uncertain parameters subject to SA:
M = 5

# Parameter ranges:
xmin = [1, 0, 0, 8, 1]
xmax = [400, 2, 1, 200, 7]

# Parameter distributions:
distr_fun = st.uniform # uniform distribution
# The shape parameters of the uniform distribution are the lower limit and the
# difference between lower and upper limits:
distr_par = [np.nan] * M
for i in range(M):
    distr_par[i] = [xmin[i], xmax[i] - xmin[i]]

# Name of parameters (will be used to customize plots):
parameter_names = ['Sm', 'beta', 'alfa', 'Rs', 'Rf']

samp_strat = "lhs"  # Latin Hypercube sampling

**History matching**

In [3]:
try:
    # Set number of sample sets
    #N_samples =
    # Set precipitation ensemble size 
    #N_precip =
    # Set minimum of precipitation multiplier (e.g. 0.8)
    #multiplier_min =
    # Set maximum of precipitation multiplier (e.g. 1.2)
    #multiplier_max =
    # Set threshold for implausibility (e.g. 3)
    #implausibility_threshold = 
    #print(f"Running the model {N_samples*N_precip} times: {N_samples} parameter sets, varying precipitation {N_precip} times")

    # Sample parameter sets using AAT_sampling (using random uniform sampling - "rsu")

    
    # Create two empty lists where we collect plausible parameter sets and plausible simulation outputs (runoff)
    # [1]
    # [2]
    
    # loop through the parameter sets
    #for  in :
        # create empty list where we collect the simulated runoff for current parameter set, with varied precipitation
        # [3]
    
        # Create precipitation multipliers between min and max (size=N_precip). 
        # You can use np.random.uniform(), check the docu to learn how to use it

        
        # loop throguh precipitation multipliers
        # for  in :
            # prepare parameters for the HyMod implementation: 1/Rf and 1/Rs, the other parameters stay the same
            #param = np.array([parameter_set[0], parameter_set[1], parameter_set[2], 1/parameter_set[3], 1/parameter_set[4]])
            
            # multiply precipitation with the current multiplier
    
            
            # run HyMod using the multiplied precipitation data ( Q, states, fluxes = HyMod.hymod_sim(param, P_multiplied, pet) )

    
            # append resulting simulated runoff for current parameter set with current precipitation to empty list defined above: at [3]
    

        # use np.array() to transform list from [3] to numpy array
    

        # Calculate the expected value of the simulated runoff for different precipitation data. You can use np.mean(). 
        # Make sure you choose the right axis to compute the mean along: the result should have the length of simulated timesteps (365 days)

    
        # Calculate the mean of the residuals of each timestep using the expected value and Q_obs[n_days:]. 
        # [n_days:] is used to crop the Q_obs data to the evaluation period 
    
    
        # Calculate the mean of the square root of the variance of the simulated runoff. 
        # You can use np.mean(npsqrt(np.var(<variable>))) 
    

        # Calculate the implausibility: divide the residual by the variance term

    
        #print(implausibility)
    
        # Check whether implausibility is too large for this parameter set (if smaller than the implausibility threshold defined above)

    
            # if plausible: append expected value of the simulated runoff for different precipitation data to list [2] and...

    
            # ... the parameter set that got plausible results to list [1]

    

    # Transform list to array (np.array()), transpose array (.T) and make a DataFrame of the result (pd.DataFrame)
    #df_Q_sims_plausible = pd.DataFrame(np.array(<list [2]>).T)  

    """
    # Preparing lineplots
    df_obs_eval = df_obs[n_days:]
    # Add date column to plausible data
    df_Q_sims_plausible["Date"] = df_obs["Date"].to_numpy()[n_days:]
    # Set date as index. This way, each column will be associated with the date when we melt the dataframe in the next step
    df_Q_sims_plausible = df_Q_sims_plausible.set_index("Date")
    # Data is in multiple columns (one for each plausible parameter set), but for plotting we need it in one column
    # We do not ignore the index in melt so that the date will remain associated with the respective runoffs of the plausible parameter sets
    df_Q_sims_plausible = df_Q_sims_plausible.melt(var_name="parameter set id", value_name="Q_sim [mm/day]", ignore_index=False)
    # Set index (is currently the date) as column
    df_Q_sims_plausible = df_Q_sims_plausible.reset_index()
    #print(f"df_Q_sims_plausible: {df_Q_sims_plausible}")    
    
    print("Plotting plausible runoff and observations")
    fig, ax = plt.subplots(1, 1, figsize=(20, 4))
    sns.lineplot(df_Q_sims_plausible, x="Date", y="Q_sim [mm/day]", linewidth=0, errorbar=("ci", 100))
    sns.lineplot(df_obs[n_days:], x="Date", y="Q [mm/day]")
    
    # Show only the main ticks
    locator = mdate.MonthLocator()
    plt.gca().xaxis.set_major_locator(locator)
    ax.set_title(catchment_name)
    plt.xticks(rotation=25)
    #plt.ylim(bottom=0, top=2)  # you may set y limits to get a clearer view on the hydrograph
    plt.show()"""
    
except Exception as error:
    print("An error occurred:", error)